In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

import keras
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense, Input, Activation, Flatten, Add
from keras.layers import BatchNormalization
# from keras.layers import GlobalAveragePooling2D
# from keras.optimizers import Adam, SGD
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D

from keras import backend

Using TensorFlow backend.


In [3]:
(xx_train, yy_train), (x_test, y_test) = cifar10.load_data()
xx_train = xx_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(xx_train, axis=0)
xx_train -= mean
x_test -= mean

x_train = xx_train[:40000]
y_train = yy_train[:40000]
x_valid = xx_train[40000:50000]
y_valid = yy_train[40000:50000]

x_train = x_train/255.0
x_valid = x_valid/255.0
x_test = x_test/255.0

y_train = np_utils.to_categorical(y_train)
y_valid = np_utils.to_categorical(y_valid)
y_test = np_utils.to_categorical(y_test)

In [66]:
x_train.shape[1:]

(32, 32, 3)

In [4]:
def add(a,b):
    shape1 = backend.int_shape(a)
    shape2 = backend.int_shape(b)
    w = int(round(shape1[1]/shape2[1]))
    h = int(round(shape1[2]/shape2[2]))
    eq = shape1[3] == shape2[3]
    
    tmp = a
    print w,h,eq
    print shape1, shape2
    if w>1 or h>1 or not eq:
        tmp = Conv2D(filters=shape2[3],kernel_size=(1,1),strides=(w,h),padding='valid',kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(a)
    print backend.int_shape(tmp)
    print
    return Add()([tmp, b])

In [5]:
def layer(num_filt, size, strides, inp):
    tmp = BatchNormalization(axis=3)(inp)
    tmp = Activation('relu')(tmp)
    tmp = Conv2D(num_filt, size, strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(tmp)
    return tmp

l = Input(x_train.shape[1:])

x = Conv2D(128, (7,7), strides=2, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(l)
x = BatchNormalization(axis=3)(x)
x = Activation('relu')(x)

#first layer
num = 32
xtmp = Conv2D(num, (1,1), strides=1, 
              padding='same', 
              kernel_initializer='he_normal', 
              kernel_regularizer=l2(0.0001))(x)
xtmp = layer(num, (3,3), 1, xtmp)
xtmp = layer(num*4, (1,1), 1, xtmp)

x = add(x,xtmp)

#other layer
xtmp = layer(num, (1,1), 1, x)
xtmp = layer(num, (3,3), 1, xtmp)
xtmp = layer(num*4, (1,1), 1, xtmp)
x = add(x,xtmp)

for i in xrange(9):
    xtmp = layer(num, (1,1), 1, x)
    xtmp = layer(num, (3,3), 1, xtmp)
    xtmp = layer(num*4, (1,1), 1, xtmp)
    x = add(x,xtmp)

    #other layer
    xtmp = layer(num, (1,1), 1, x)
    xtmp = layer(num, (3,3), 1, xtmp)
    xtmp = layer(num*4, (1,1), 1, xtmp)
    x = add(x,xtmp)

x = BatchNormalization()(x)
x = Activation('relu')(x)

xshape = backend.int_shape(x)
x = AveragePooling2D(pool_size=(xshape[1],xshape[2]), strides=(1,1))(x)
x = Flatten()(x)

x = Dense(10, activation='softmax', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)

early=EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='auto')
filepath="weights.best.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None

In [46]:
def layer(num_filt, size, strides, inp):
    tmp = BatchNormalization(axis=3)(inp)
    tmp = Activation('relu')(tmp)
    tmp = Conv2D(num_filt, size, strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(tmp)
    return tmp

l = Input(x_train.shape[1:])

x = Conv2D(64, (7,7), strides=2, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(l)
x = BatchNormalization(axis=3)(x)
x = Activation('relu')(x)

#first layer
num = 64
xtmp = Conv2D(num, (1,1), strides=1, 
              padding='same', 
              kernel_initializer='he_normal', 
              kernel_regularizer=l2(0.0001))(x)
xtmp = layer(num, (3,3), 1, xtmp)
xtmp = layer(num*4, (1,1), 1, xtmp)

x = add(x,xtmp)

#other layer
xtmp = layer(num, (1,1), 1, x)
xtmp = layer(num, (3,3), 1, xtmp)
xtmp = layer(num*4, (1,1), 1, xtmp)
x = add(x,xtmp)

for i in xrange(3):
    num *= 2
    xtmp = layer(num, (1,1), 2, x)
    xtmp = layer(num, (3,3), 1, xtmp)
    xtmp = layer(num*4, (1,1), 1, xtmp)
    x = add(x,xtmp)

    #other layer
    xtmp = layer(num, (1,1), 1, x)
    xtmp = layer(num, (3,3), 1, xtmp)
    xtmp = layer(num*4, (1,1), 1, xtmp)
    x = add(x,xtmp)

x = BatchNormalization()(x)
x = Activation('relu')(x)

xshape = backend.int_shape(x)
x = AveragePooling2D(pool_size=(xshape[1],xshape[2]), strides=(1,1))(x)
x = Flatten()(x)

x = Dense(10, activation='softmax', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)

early=EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='auto')
filepath="weights.best.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

1 1 False
(None, 16, 16, 64) (None, 16, 16, 256)
(None, 16, 16, 256)

1 1 True
(None, 16, 16, 256) (None, 16, 16, 256)
(None, 16, 16, 256)

2 2 False
(None, 16, 16, 256) (None, 8, 8, 512)
(None, 8, 8, 512)

1 1 True
(None, 8, 8, 512) (None, 8, 8, 512)
(None, 8, 8, 512)

2 2 False
(None, 8, 8, 512) (None, 4, 4, 1024)
(None, 4, 4, 1024)

1 1 True
(None, 4, 4, 1024) (None, 4, 4, 1024)
(None, 4, 4, 1024)

2 2 False
(None, 4, 4, 1024) (None, 2, 2, 2048)
(None, 2, 2, 2048)

1 1 True
(None, 2, 2, 2048) (None, 2, 2, 2048)
(None, 2, 2, 2048)



In [43]:
def layer(num_filt, strides, inp):
    tmp = BatchNormalization(axis=3)(inp)
    tmp = Activation('relu')(tmp)
    tmp = Conv2D(num_filt, (3,3), strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(tmp)
    return tmp

l = Input(x_train.shape[1:])

x = Conv2D(64, (7,7), strides=2, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(l)
x = BatchNormalization(axis=3)(x)
x = Activation('relu')(x)

#first layer
num = 64
xtmp = Conv2D(num, (3,3), strides=1, 
              padding='same', 
              kernel_initializer='he_normal', 
              kernel_regularizer=l2(0.0001))(x)
xtmp = layer(num, 1, xtmp)
x = add(x,xtmp)

#other layer
xtmp = layer(num, 1, x)
xtmp = layer(num, 1, xtmp)
x = add(x,xtmp)

for i in xrange(3):
    num *= 2
    xtmp = layer(num, 2, x)
    xtmp = layer(num, 1, xtmp)
    x = add(x,xtmp)
    
    #other layer
    xtmp = layer(num, 1, x)
    xtmp = layer(num, 1, xtmp)
    x = add(x,xtmp)

x = BatchNormalization()(x)
x = Activation('relu')(x)

xshape = backend.int_shape(x)
x = AveragePooling2D(pool_size=(xshape[1],xshape[2]), strides=(1,1))(x)
x = Flatten()(x)

x = Dense(10, activation='softmax', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)

early=EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='auto')
filepath="weights.best.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

1 1 True
(None, 16, 16, 64) (None, 16, 16, 64)
(None, 16, 16, 64)

1 1 True
(None, 16, 16, 64) (None, 16, 16, 64)
(None, 16, 16, 64)

2 2 False
(None, 16, 16, 64) (None, 8, 8, 128)
(None, 8, 8, 128)

1 1 True
(None, 8, 8, 128) (None, 8, 8, 128)
(None, 8, 8, 128)

2 2 False
(None, 8, 8, 128) (None, 4, 4, 256)
(None, 4, 4, 256)

1 1 True
(None, 4, 4, 256) (None, 4, 4, 256)
(None, 4, 4, 256)

2 2 False
(None, 4, 4, 256) (None, 2, 2, 512)
(None, 2, 2, 512)

1 1 True
(None, 2, 2, 512) (None, 2, 2, 512)
(None, 2, 2, 512)



In [6]:
model = Model(l,x)

model.compile(
    optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 128)  18944       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 128)  512         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 16, 16, 128)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [7]:
#if training was cut short
model = load_model('weights.best.hdf5')

In [ ]:
history = model.fit(x_train,y_train,
          batch_size=32, epochs=200,
          shuffle=True,
          validation_data=(x_valid, y_valid), 
          callbacks=[checkpoint, early])

Train on 40000 samples, validate on 10000 samples
Epoch 1/200
40000/40000 [==============================] - 1570s 39ms/step - loss: 1.0931 - acc: 0.7000 - val_loss: 1.2910 - val_acc: 0.6294
Epoch 2/200
40000/40000 [==============================] - 1530s 38ms/step - loss: 0.9959 - acc: 0.7351 - val_loss: 1.0992 - val_acc: 0.7028
Epoch 3/200
40000/40000 [==============================] - 26298s 657ms/step - loss: 0.9328 - acc: 0.7565 - val_loss: 1.1457 - val_acc: 0.6827
Epoch 4/200
40000/40000 [==============================] - 2248s 56ms/step - loss: 0.8837 - acc: 0.7736 - val_loss: 1.0588 - val_acc: 0.7139
Epoch 5/200
40000/40000 [==============================] - 1758s 44ms/step - loss: 0.8367 - acc: 0.7915 - val_loss: 1.2781 - val_acc: 0.6547
Epoch 6/200
40000/40000 [==============================] - 1991s 50ms/step - loss: 0.8046 - acc: 0.8028 - val_loss: 1.0729 - val_acc: 0.7223
Epoch 7/200
 6464/40000 [===>..........................] - ETA: 23:35 - loss: 0.7486 - acc: 0.8204

In [35]:
res = model.evaluate(x_test, y_test, batch_size=512)

10000/10000 [==============================] - 25s 3ms/step


In [36]:
print('Loss: %.2f Accuracy: %.2f%%' % (res[0], res[1]*100.0))

Loss: 1.71 Accuracy: 67.62%
